In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from terminal_optimization import container_objects
from terminal_optimization import container_defaults
from terminal_optimization import container_system

NameError: name 'containers_mixins' is not defined

# 0. Prepare inputs

In [ ]:
startyear = 2018
lifecycle = 10

## 0.1 Generate demand forecast scenario

In [ ]:
years = list(range(startyear, startyear+lifecycle))

demand = []
for year in years:
    if year < 2023:
        demand.append(750_000)
    else:
        demand.append(1_600_000)

scenario_data={'year': years, 'volume': demand}

# instantiate a commodity objects
# maize
container_defaults.maize_data['handysize_perc'] = 0
container_defaults.maize_data['handymax_perc'] = 100
container_defaults.maize_data['panamax_perc'] = 0
container_defaults.maize_data['handling_fee'] = 9.8 # example Wijnand
container_defaults.maize_data['historic_data'] = []
maize = container_objects.Commodity(**container_defaults.maize_data)
maize.scenario_data = pd.DataFrame(data=scenario_data)

# wheat
container_defaults.wheat_data['historic_data'] = []
container_defaults.wheat_data['scenario_data'] = []
wheat = container_objects.Commodity(**container_defaults.wheat_data)

# soybeans
container_defaults.soybean_data['historic_data'] = []
container_defaults.soybean_data['scenario_data'] = []
soybeans = container_objects.Commodity(**container_defaults.soybean_data)

# combine
demand = [maize, wheat, soybeans]

# plot scenarios 
if False:
    maize.plot_demand()
    wheat.plot_demand()
    soybeans.plot_demand()

## 0.2 Generate vessels

In [ ]:
# instantiate vessels
handysize = container_objects.Vessel(**container_defaults.handysize_data)
container_defaults.handymax_data['call_size'] = 50000
handymax = container_objects.Vessel(**container_defaults.handymax_data)
panamax = container_objects.Vessel(**container_defaults.panamax_data)


vessels = [handysize, handymax, panamax] 

# 1. Instatiate terminal system

In [ ]:
Terminal = container_system.System(startyear = startyear, 
                         lifecycle = lifecycle, 
                         elements = demand + vessels, 
                         operational_hours = 5840,   # example Wijnand
                         debug=True,
                         crane_type_defaults=container_defaults.mobile_crane_data, 
                         storage_type_defaults=container_defaults.silo_data)

# 2. Start simulation
Develop terminal given the throughput scenario and investment triggers implemented

In [ ]:
Terminal.simulate()

## Report all elements

In [ ]:
if False: 
    for element in Terminal.elements:
        print("")
        print(element.name)
        print("")
        print(element.__dict__)

In [ ]:
Terminal.terminal_elements_plot()

In [ ]:
Terminal.terminal_capacity_plot()

In [ ]:
# add cash flow information for each of the Terminal elements
cash_flows, cash_flows_WACC_real = Terminal.add_cashflow_elements()

In [ ]:
Terminal.cashflow_plot(cash_flows)

In [ ]:
Terminal.cashflow_plot(cash_flows_WACC_real)

## SandBox

In [ ]:
Terminal.NPV()